#### Draw pRFx refit plots 

In [ ]:
# Imports
import os
import numpy as np
import nibabel as nb
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# Figure imports
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.express as px
from plot_utils import plotly_template

# stats
n_permutation = 10000
from scipy.stats import permutation_test
def statistic(condA, condB, axis):
    return np.nanmean(condA, axis=axis) - np.nanmean(condB, axis=axis)

# Define folders
base_dir = '/home/mszinte/disks/meso_S/data/gaze_prf'
bids_dir = "{}".format(base_dir)
pp_dir = "{}/derivatives/pp_data".format(base_dir)

# settings
subjects = ['sub-001', 'sub-002', 'sub-003', 'sub-004',
            'sub-005', 'sub-006', 'sub-007', 'sub-008']
subjects_plot = ['sub-001', 'sub-002', 'sub-003', 'sub-004',
                 'sub-005', 'sub-006', 'sub-007', 'sub-008', 'group']
tasks = ['FullScreen', 'FullScreenAttendFix', 'FullScreenAttendBar']
rois = ['V1', 'V2', 'V3', 'V3AB', 'hMT+', 'LO',
        'VO', 'iIPS', 'sIPS', 'iPCS', 'sPCS', 'mPCS']

best_voxels_num = 250
type_analyses = ['','_best{}'.format(best_voxels_num)]

# general figure settings
template_specs = dict(  axes_color="rgba(0, 0, 0, 1)",
                        axes_width=2,
                        axes_font_size=13,
                        bg_col="rgba(255, 255, 255, 1)",
                        font='Arial',
                        title_font_size=15,
                        plot_width=1.5)

fig_template = plotly_template(template_specs)

In [ ]:
# Subplot settings
left_shift, right_shift = -4.0, 4.0
show_indiv = True
margin_t, margin_b, margin_l, margin_r = 50, 100 ,100 ,50
rows, cols = 2, 2
column_widths,row_heights = [1,1], [1,1]
sb_specs = [[{},{}],[{},{}]]
subplot_width, subplot_height = 300, 300

fig_width, fig_height = 0, 0
for column_width in column_widths:fig_width += subplot_width*column_width
for row_height in row_heights: fig_height +=subplot_height*row_height
fig_width = fig_width + margin_l + margin_r
fig_height = fig_height + margin_t + margin_b

line_width = 2
xaxis_range = [-8, 8]
xaxis_tick = 5
xaxis_tickvals = np.linspace(xaxis_range[0],xaxis_range[1],xaxis_tick)
xaxis_title = 'Gaze center<br><i>pRFx</i> (<i>screen</i> dva)'
xaxis_ticktext = []
[xaxis_ticktext.append(('{:g}'.format(val))) for val in xaxis_tickvals]

yaxis_range = [-8, 8]
yaxis_tick = 5
yaxis_tickvals = np.linspace(yaxis_range[0],yaxis_range[1],xaxis_tick)
yaxis_title = 'Gaze left/right<br><i>pRFx</i> (<i>screen</i> dva)'
yaxis_ticktext = []
[yaxis_ticktext.append(('{:g}'.format(val))) for val in yaxis_tickvals]

gr_ab_line_color, gr_af_line_color = 'rgba(143, 20, 85, 1)', 'rgba(230, 151, 193, 1)'
gl_ab_line_color, gl_af_line_color = 'rgba(41, 102, 45, 1)', 'rgba(153, 198, 98, 1)'
gr_ab_eb_color, gr_af_eb_color = 'rgba(143, 20, 85, .5)', 'rgba(230, 151, 193, .5)'
gl_ab_eb_color, gl_af_eb_color = 'rgba(41, 102, 45, .5)', 'rgba(153, 198, 98, .5)'

gr_ab_line_color, gr_af_line_color = 'rgba(143, 20, 85, 1)', 'rgba(230, 151, 193, 1)'
gl_ab_line_color, gl_af_line_color = 'rgba(41, 102, 45, 1)', 'rgba(153, 198, 98, 1)'
gr_ab_eb_color, gr_af_eb_color = 'rgba(143, 20, 85, .5)', 'rgba(230, 151, 193, .5)'
gl_ab_eb_color, gl_af_eb_color = 'rgba(41, 102, 45, .5)', 'rgba(153, 198, 98, .5)'

hover_left = '<i>gaze center</i>: %{x:1.2f}<br>' + '<i>gaze left</i>: %{y:1.2f}'
hover_right = '<i>gaze center</i>: %{x:1.2f}<br>' + '<i>gaze right</i>: %{y:1.2f}'

groups_rois = [['V1', 'hMT+'], ['V1', 'V2'], ['V3', 'V3AB'], ['hMT+', 'LO'],
               ['VO', 'iIPS'], ['sIPS', 'iPCS'], ['sPCS', 'mPCS']]

for type_analysis in type_analyses:    
    # load TSV
    for subject_num, subject in enumerate(subjects):
        tsv_dir = '{}/{}/prf/tsv'.format(pp_dir, subject)
        df_fn = "{}/{}_refit_res{}.pkl".format(tsv_dir,subject,type_analysis)
        df_sub = pd.read_pickle(df_fn)

        # create new df
        df_new = pd.DataFrame({'subject': df_sub.subject}) 
        df_new['roi'] = df_sub.roi

        df_new['gc_af_optim_x'] = df_sub.gc_af_optim_x
        df_new['gl_af_optim_x'] = df_sub.gl_af_optim_x
        df_new['gr_af_optim_x'] = df_sub.gr_af_optim_x
        df_new['gc_ab_optim_x'] = df_sub.gc_ab_optim_x 
        df_new['gl_ab_optim_x'] = df_sub.gl_ab_optim_x
        df_new['gr_ab_optim_x'] = df_sub.gr_ab_optim_x

        # across roi
        if subject_num > 0:  df = pd.concat([df,df_new], ignore_index=True)
        else: df = df_new

    df.roi = pd.Categorical(df.roi,categories=rois)

    # compute bins
    range_vals = np.linspace(-8.5, 8.5, 18)
    range_labels = np.linspace(-8.0, 8.0, 17)
    df['gc_af_optim_x_bin']=pd.cut(x=df['gc_af_optim_x'],bins=range_vals, labels=range_labels)
    df['gc_ab_optim_x_bin']=pd.cut(x=df['gc_ab_optim_x'],bins=range_vals, labels=range_labels)


    #for subject in subjects_plot:
    for subject in ['group']:
        tsv_dir = '{}/{}/prf/tsv'.format(pp_dir, subject)
        subplot_titles = ['<b>Attend-bar</b><br>({})'.format(subject),
                          '<b>Attend-fix</b><br>({})'.format(subject)]

        for group_rois in groups_rois:

            fig = make_subplots(rows=rows, cols=cols, specs=sb_specs, print_grid=False, vertical_spacing=0.2, horizontal_spacing=0.2, 
                            column_widths=column_widths, row_heights=row_heights,  subplot_titles=subplot_titles)

            for col, attend_task in enumerate(['ab','af']):

                for row, roi in enumerate(group_rois):

                    # pick color
                    exec("gl_line_color, gr_line_color = gl_{}_line_color, gr_{}_line_color".format(attend_task, attend_task))
                    exec("gl_eb_color, gr_eb_color = gl_{}_eb_color, gr_{}_eb_color".format(attend_task, attend_task))

                    # get data
                    if subject == 'group':
                        # gaze center vs. left
                        x_gl_df = df.groupby(['roi','subject','gc_{}_optim_x_bin'.format(attend_task)])['gc_{}_optim_x'.format(attend_task)].mean().reset_index(name='gc_{}_optim_x_mean'.format(attend_task))\
                                              .groupby(['roi','gc_{}_optim_x_bin'.format(attend_task)])['gc_{}_optim_x_mean'.format(attend_task)].mean().reset_index(name='gc_{}_optim_x_mean'.format(attend_task)) 
                        x_gl = x_gl_df.loc[x_gl_df.roi==roi]['gc_{}_optim_x_mean'.format(attend_task)]

                        y_gl_df = df.groupby(['roi','subject','gc_{}_optim_x_bin'.format(attend_task)])['gl_{}_optim_x'.format(attend_task)].mean().reset_index(name='gl_{}_optim_x_mean'.format(attend_task))\
                                            .groupby(['roi','gc_{}_optim_x_bin'.format(attend_task)])['gl_{}_optim_x_mean'.format(attend_task)].mean().reset_index(name='gl_{}_optim_x_mean'.format(attend_task)) 
                        y_gl = y_gl_df.loc[y_gl_df.roi==roi]['gl_{}_optim_x_mean'.format(attend_task)]

                        eb_gl_df = df.groupby(['roi','subject','gc_{}_optim_x_bin'.format(attend_task)])['gl_{}_optim_x'.format(attend_task)].mean().reset_index(name='gl_{}_optim_x_mean'.format(attend_task))\
                                            .groupby(['roi','gc_{}_optim_x_bin'.format(attend_task)])['gl_{}_optim_x_mean'.format(attend_task)].sem(ddof=-1).reset_index(name='gl_{}_optim_x_mean'.format(attend_task)) 
                        eb_gl = eb_gl_df.loc[eb_gl_df.roi==roi]['gl_{}_optim_x_mean'.format(attend_task)]

                        # gaze center vs. left
                        x_gr_df = df.groupby(['roi','subject','gc_{}_optim_x_bin'.format(attend_task)])['gc_{}_optim_x'.format(attend_task)].mean().reset_index(name='gc_{}_optim_x_mean'.format(attend_task))\
                                            .groupby(['roi','gc_{}_optim_x_bin'.format(attend_task)])['gc_{}_optim_x_mean'.format(attend_task)].mean().reset_index(name='gc_{}_optim_x_mean'.format(attend_task))
                        x_gr = x_gr_df.loc[x_gr_df.roi==roi]['gc_{}_optim_x_mean'.format(attend_task)]

                        y_gr_df = df.groupby(['roi','subject','gc_{}_optim_x_bin'.format(attend_task)])['gr_{}_optim_x'.format(attend_task)].mean().reset_index(name='gr_{}_optim_x_mean'.format(attend_task))\
                                            .groupby(['roi','gc_{}_optim_x_bin'.format(attend_task)])['gr_{}_optim_x_mean'.format(attend_task)].mean().reset_index(name='gr_{}_optim_x_mean'.format(attend_task))
                        y_gr = y_gr_df.loc[y_gr_df.roi==roi]['gr_{}_optim_x_mean'.format(attend_task)]

                        eb_gr_df = df.groupby(['roi','subject','gc_{}_optim_x_bin'.format(attend_task)])['gr_{}_optim_x'.format(attend_task)].mean().reset_index(name='gr_{}_optim_x_mean'.format(attend_task))\
                                            .groupby(['roi','gc_{}_optim_x_bin'.format(attend_task)])['gr_{}_optim_x_mean'.format(attend_task)].sem(ddof=-1).reset_index(name='gr_{}_optim_x_mean'.format(attend_task))
                        eb_gr = eb_gr_df.loc[eb_gr_df.roi==roi]['gr_{}_optim_x_mean'.format(attend_task)]

                    else:
                        # gaze center vs. left
                        x_gl = df.loc[(df.subject==subject)&(df.roi==roi)].groupby(df['gc_{}_optim_x_bin'.format(attend_task)])['gc_{}_optim_x'.format(attend_task)].mean()
                        y_gl = df.loc[(df.subject==subject)&(df.roi==roi)].groupby(df['gc_{}_optim_x_bin'.format(attend_task)])['gl_{}_optim_x'.format(attend_task)].mean()
                        eb_gl = df.loc[(df.subject==subject)&(df.roi==roi)].groupby(df['gc_{}_optim_x_bin'.format(attend_task)])['gl_{}_optim_x'.format(attend_task)].std()

                        # gaze center vs. left
                        x_gr = df.loc[(df.subject==subject)&(df.roi==roi)].groupby(df['gc_{}_optim_x_bin'.format(attend_task)])['gc_{}_optim_x'.format(attend_task)].mean()
                        y_gr = df.loc[(df.subject==subject)&(df.roi==roi)].groupby(df['gc_{}_optim_x_bin'.format(attend_task)])['gr_{}_optim_x'.format(attend_task)].mean()
                        eb_gr = df.loc[(df.subject==subject)&(df.roi==roi)].groupby(df['gc_{}_optim_x_bin'.format(attend_task)])['gr_{}_optim_x'.format(attend_task)].std()

                    # plots
                    fig.append_trace(go.Scatter(x=x_gl, y=y_gl+left_shift, showlegend=False, mode='lines', line_color=gl_line_color, line_width=line_width, connectgaps=True, name='', hovertemplate=hover_left), row=row+1, col=col+1)
                    fig.append_trace(go.Scatter(x=x_gl, y=y_gl+left_shift+eb_gl, showlegend=False, mode='lines', fillcolor=gl_eb_color, line_width=0, connectgaps=True, hoverinfo='skip'), row=row+1, col=col+1)
                    fig.append_trace(go.Scatter(x=x_gl, y=y_gl+left_shift-eb_gl, showlegend=False, mode='lines', fillcolor=gl_eb_color, line_width=0, connectgaps=True, hoverinfo='skip', fill='tonexty'), row=row+1, col=col+1)

                    fig.append_trace(go.Scatter(x=x_gr, y=y_gr+right_shift, showlegend=False, mode='lines', line_color=gr_line_color, line_width=line_width, connectgaps=True, name='', hovertemplate=hover_right), row=row+1, col=col+1)
                    fig.append_trace(go.Scatter(x=x_gr, y=y_gr+right_shift+eb_gr, showlegend=False, mode='lines', fillcolor=gr_eb_color, line_width=0, connectgaps=True, hoverinfo='skip'), row=row+1, col=col+1)
                    fig.append_trace(go.Scatter(x=x_gr, y=y_gr+right_shift-eb_gr, showlegend=False, mode='lines', fillcolor=gr_eb_color, line_width=0, connectgaps=True, hoverinfo='skip', fill='tonexty'), row=row+1, col=col+1)

            # anotation + legend
            for xref, yref, roi_label, gl_line_color, gr_line_color, legrow, legcol in zip(['x1','x2','x3','x4'],['y1','y2','y3','y4'],[group_rois[0],group_rois[0],group_rois[1],group_rois[1]],
                                                                                           [gl_ab_line_color,gl_af_line_color,gl_ab_line_color,gl_af_line_color],
                                                                                           [gr_ab_line_color,gr_af_line_color,gr_ab_line_color,gr_af_line_color],
                                                                                           [1,1,2,2], [1,2,1,2]):
                fig.add_annotation(x=6, y=-6, xref=xref, yref=yref, showarrow=False, text='<b>{}</b>'.format(roi_label))
                fig.add_annotation(x=-6.5, y=6.7, xref=xref, yref=yref, showarrow=False, text='<i>gaze left</i>', xanchor='left', yanchor='middle')
                fig.add_annotation(x=-6.5, y=5.5, xref=xref, yref=yref, showarrow=False, text='<i>gaze right</i>',xanchor='left', yanchor='middle')
                fig.append_trace(go.Scatter(x=np.linspace(-7.5,-6.7,5), y=np.linspace(-7.5,-6.7,5)*0+6.6, showlegend=False,line_color=gl_line_color, line_width=line_width, mode='lines'),row=legrow, col=legcol)
                fig.append_trace(go.Scatter(x=np.linspace(-7.5,-6.7,5), y=np.linspace(-7.5,-6.7,5)*0+5.4, showlegend=False,line_color=gr_line_color, line_width=line_width, mode='lines'),row=legrow, col=legcol)

            # set axis
            for row in np.arange(rows):
                for col in np.arange(cols):
                    fig.update_xaxes(visible=True, ticklen=8, linewidth=template_specs['axes_width'], row=row+1, col=col+1)
                    fig.update_yaxes(visible=True, ticklen=8, linewidth=template_specs['axes_width'], row=row+1, col=col+1)

            # set figure axis
            for xaxis in ['xaxis','xaxis2','xaxis3','xaxis4']:
                exec("fig.layout.update({}_range=xaxis_range)".format(xaxis))
                exec("fig.layout.update({}_title=xaxis_title)".format(xaxis))
                exec("fig.layout.update({}_tickvals=xaxis_tickvals)".format(xaxis))
                exec("fig.layout.update({}_ticktext=xaxis_ticktext)".format(xaxis))

            for yaxis in ['yaxis','yaxis2','yaxis3','yaxis4']:
                exec("fig.layout.update({}_range=yaxis_range)".format(yaxis))
                exec("fig.layout.update({}_title=yaxis_title)".format(yaxis))
                exec("fig.layout.update({}_tickvals=yaxis_tickvals)".format(yaxis))
                exec("fig.layout.update({}_ticktext=yaxis_ticktext)".format(yaxis))
                exec("fig.layout.update({}_title_standoff=10)".format(yaxis))

            fig.layout.update(template=fig_template, width=fig_width, height=fig_height, margin_l=margin_l, 
                              margin_r=margin_r, margin_t=margin_t, margin_b=margin_b)

            # show and save figure
            if show_indiv == True:
                fig.show(config={"displayModeBar": False})
            else:
                if subject == 'group':fig.show(config={"displayModeBar": False})

            fn = "{}/{}_refit_{}_{}{}".format(tsv_dir, subject, group_rois[0], group_rois[1], type_analysis)
            print('{}.pdf'.format(fn))
            fig.write_image('{}.pdf'.format(fn))
            print('{}.html'.format(fn))
            fig.write_html('{}.html'.format(fn),config={"displayModeBar": False})